# Module 3. Deployment on MMS(Multi Model Server)
---

본 모듈에서는 모델의 배포(deployment)를 수행합니다. 노트북 실행에는 약 15분 가량 소요되며, 핸즈온 실습 시에는 25분을 권장드립니다.

<br>

## 1. Inference script
---

아래 코드 셀은 `src` 디렉토리에 SageMaker 추론 스크립트인 `inference.py`를 저장합니다.<br>

이 스크립트는 SageMaker 상에서 MMS(Multi Model Server)를 쉽고 편하게 배포할 수 이는 high-level 툴킷인 SageMaker inference toolkit의 인터페이스를
사용하고 있으며, 여러분께서는 인터페이스에 정의된 핸들러(handler) 함수들만 구현하시면 됩니다.

#### MMS(Multi Model Server)란?
- [https://github.com/awslabs/multi-model-server](https://github.com/awslabs/multi-model-server) (2017년 12월 초 MXNet 1.0 릴리스 시 최초 공개, MXNet용 모델 서버로 시작)
- Prerequisites: Java 8, MXNet (단, MXNet 사용 시에만)
- MMS는 프레임워크에 구애받지 않도록 설계되었기 때문에, 모든 프레임워크의 백엔드 엔진 역할을 할 수 있는 충분한 유연성을 제공합니다.
- SageMaker MXNet 추론 컨테이너와 PyTorch 추론 컨테이너는 SageMaker inference toolkit으로 MMS를 래핑하여 사용합니다.
    - 2020년 4월 말 PyTorch용 배포 웹 서비스인 torchserve가 출시되면서, 향후 PyTorch 추론 컨테이너는 MMS 기반에서 torchserve 기반으로 마이그레이션될 예정입니다. 

In [1]:
%%writefile ./src/inference.py

import os
import pandas as pd
import gluonts 
import numpy as np
import argparse
import json
import pathlib
from mxnet import gpu, cpu
from mxnet.context import num_gpus
import matplotlib.pyplot as plt

from gluonts.dataset.util import to_pandas
from gluonts.distribution import DistributionOutput, StudentTOutput, NegativeBinomialOutput, GaussianOutput
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions, backtest_metrics
from gluonts.model.predictor import Predictor
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.common import ListDataset


def model_fn(model_dir):
    path = pathlib.Path(model_dir)   
    predictor = Predictor.deserialize(path)
    print("model was loaded successfully")
    return predictor


def transform_fn(model, request_body, content_type='application/json', accept_type='application/json'):
    
    related_cols = ['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main', 'weather_description']    
    FREQ = 'H'
    pred_length = 24*7

    data = json.loads(request_body)

    target_test_df = pd.DataFrame(data['target_values'], index=data['timestamp'])
    related_test_df = pd.DataFrame(data['related_values'], index=data['timestamp'])
    related_test_df.columns = related_cols
        
    target = target_test_df.values
    related = related_test_df.values
    num_series = target_test_df.shape[1]
    start_dt = target_test_df.index[0]

    related_list = [related_test_df[c].values for c in related_cols]    
    test_lst = []

    target_vec = target.squeeze()
    related_vecs = [related.squeeze() for related in related_list]
    dic = {FieldName.TARGET: target_vec, 
           FieldName.START: start_dt,
           FieldName.FEAT_DYNAMIC_REAL: related_vecs
          } 
    test_lst.append(dic)
    test_ds = ListDataset(test_lst, freq=FREQ)
    
    response_body = {}
    forecast_it = model.predict(test_ds)
    forecast = list(forecast_it)
    response_body['out'] = forecast[0].samples.mean(axis=0).tolist()
    return json.dumps(response_body)

Overwriting ./src/inference.py


<br>

## 2. Test Inference code 
---

엔드포인트 배포 전, 추론 스크립트를 검증합니다. 

In [2]:
%store -r

In [3]:
from src.inference import model_fn, transform_fn
import json
import numpy as np
import pandas as pd

# Prepare test data
target_test_df = pd.read_csv("data/target_test.csv", index_col=0)
related_test_df = pd.read_csv("data/related_test.csv", index_col=0)

input_data = {'target_values': target_test_df.values.tolist(), 
              'related_values': related_test_df.values.tolist(),
              'timestamp': target_test_df.index.tolist()
             }
request_body = json.dumps(input_data)

# Test inference script 
model = model_fn('./model')
response = transform_fn(model, request_body)
outputs = json.loads(response)
print(outputs['out'])

model was loaded successfully
[656.91064453125, 561.4729614257812, 347.90850830078125, 589.5120849609375, 1208.7757568359375, 2401.4052734375, 4090.120361328125, 4432.9208984375, 5057.35595703125, 2463.945556640625, 4192.39404296875, 4626.38671875, 4401.234375, 4769.94677734375, 5059.52734375, 6265.05615234375, 6696.24853515625, 6363.36669921875, 6048.70751953125, 3575.61865234375, 3003.282470703125, 2079.645263671875, 1056.0457763671875, 810.9989624023438, 425.81524658203125, 418.6568603515625, 485.02008056640625, 136.82293701171875, 1191.1549072265625, 3784.2900390625, 5694.00927734375, 6540.23681640625, 7461.63134765625, 5763.88330078125, 5628.015625, 5755.81982421875, 5388.52978515625, 5715.1064453125, 6246.8251953125, 6802.5185546875, 7542.9619140625, 7001.9814453125, 5324.79248046875, 4155.74365234375, 3336.8203125, 2572.806884765625, 1908.6240234375, 719.6004638671875, 583.6642456054688, 223.6375732421875, 7.462818622589111, 236.8397979736328, 1103.4217529296875, 3340.1164550781

<br>

## 3. Local Endpoint Inference
---

충분한 검증 및 테스트 없이 훈련된 모델을 곧바로 실제 운영 환경에 배포하기에는 많은 위험 요소들이 있습니다. 따라서, 로컬 모드를 사용하여 실제 운영 환경에 배포하기 위한 추론 인스턴스를 시작하기 전에 노트북 인스턴스의 로컬 환경에서 모델을 배포하는 것을 권장합니다. 이를 로컬 모드 엔드포인트(Local Mode Endpoint)라고 합니다.

In [4]:
import os
import time
import sagemaker
from sagemaker.mxnet import MXNetModel
role = sagemaker.get_execution_role()

In [5]:
local_model_path = f'file://{os.getcwd()}/model/model.tar.gz'
endpoint_name = "local-endpoint-traffic-volume-forecast-{}".format(int(time.time()))


아래 코드 셀을 실행 후, 로그를 확인해 보세요. MMS에 대한 세팅값들을 확인하실 수 있습니다.

```bash
algo-1-u3xwd_1  | MMS Home: /usr/local/lib/python3.6/site-packages
algo-1-u3xwd_1  | Current directory: /
algo-1-u3xwd_1  | Temp directory: /home/model-server/tmp
algo-1-u3xwd_1  | Number of GPUs: 0
algo-1-u3xwd_1  | Number of CPUs: 2
algo-1-u3xwd_1  | Max heap size: 878 M
algo-1-u3xwd_1  | Python executable: /usr/local/bin/python3.6
algo-1-u3xwd_1  | Config file: /etc/sagemaker-mms.properties
algo-1-u3xwd_1  | Inference address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Management address: http://0.0.0.0:8080
algo-1-u3xwd_1  | Model Store: /.sagemaker/mms/models
...
```

In [6]:
local_model = MXNetModel(model_data=local_model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = local_model.deploy(instance_type='local', 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Attaching to tmpr4ropmh6_algo-1-71m4l_1
algo-1-71m4l_1  | Collecting pandas==1.0.0
algo-1-71m4l_1  |   Downloading pandas-1.0.0-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
     |████████████████████████████████| 10.1 MB 26.0 MB/s eta 0:00:01
algo-1-71m4l_1  | Collecting gluonts==0.5.1
algo-1-71m4l_1  |   Downloading gluonts-0.5.1-py3-none-any.whl (419 kB)
     |████████████████████████████████| 419 kB 33.4 MB/s eta 0:00:01
algo-1-71m4l_1  | Requirement already satisfied: numpy>=1.13.3 in /usr/local/lib/python3.6/site-packages (from pandas==1.0.0->-r /opt/ml/model/code/requirements.txt (line 1)) (1.17.4)
algo-1-71m4l_1  | Requirement already satisfied: python-dateutil>=2.6.1 in /usr/local/lib/python3.6/site-packages (from pandas==1.0.0->-r /opt/ml/model/code/requirements.txt (line 1)) (2.8.1)
algo-1-71m4l_1  | Collecting pytz>=2017.2
algo-1-71m4l_1  |   Downloading pytz-2020.1-py2.py3-none-any.whl (510 kB)
     |████████████████████████████████| 510 kB 43.8 MB/s eta 0:00:01
algo-1-71m4l_

algo-1-71m4l_1  | 2020-08-31 13:59:55,820 [INFO ] pool-1-thread-6 ACCESS_LOG - /172.18.0.1:46542 "GET /ping HTTP/1.1" 200 42
!algo-1-71m4l_1  | 2020-08-31 13:59:56,732 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-71m4l_1  | 2020-08-31 13:59:56,768 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-71m4l_1  | 2020-08-31 13:59:56,770 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-71m4l_1  | 2020-08-31 13:59:56,804 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Generating new fontManager, this may take some time...
algo-1-71m4l_1  | 2020-08-31 13:59:57,789 [INFO ] W-9000-model-stdout com.amazonaws.ml.mms.wlm.WorkerLifeCycle - Using CPU
algo-1-71m4l_1  | 2020-08-31 13:59:57,822 [INFO ] W-9000-model-stdout com.amazo

로컬에서 컨테이너를 배포했기 때문에 컨테이너가 현재 실행 중임을 확인할 수 있습니다.

In [7]:
!docker ps

CONTAINER ID        IMAGE                                                                        COMMAND                  CREATED             STATUS              PORTS                              NAMES
20691673ebd5        763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-cpu-py3   "python /usr/local/b…"   32 seconds ago      Up 30 seconds       0.0.0.0:8080->8080/tcp, 8081/tcp   tmpr4ropmh6_algo-1-71m4l_1


### Inference using SageMaker SDK

SageMaker SDK의 `predict()` 메서드로 쉽게 추론을 수행할 수 있습니다. 

In [8]:
outputs = predictor.predict(input_data)

algo-1-71m4l_1  | 2020-08-31 14:00:17,993 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 27
algo-1-71m4l_1  | 2020-08-31 14:00:17,993 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:47066 "POST /invocations HTTP/1.1" 200 31


In [9]:
print(outputs)

{'out': [557.2713012695312, 603.84033203125, 355.400146484375, 594.2640380859375, 1020.9503173828125, 2260.816162109375, 5223.166015625, 5095.93505859375, 4293.2744140625, 4693.7001953125, 4462.06982421875, 4440.93359375, 4481.41357421875, 4553.86474609375, 5249.0595703125, 7149.01611328125, 7378.599609375, 6582.26708984375, 5148.14111328125, 3448.327392578125, 2328.8837890625, 2421.5009765625, 1240.4501953125, 969.880615234375, 356.9651184082031, 410.36956787109375, 498.94891357421875, 158.7935333251953, 1257.3603515625, 3725.76708984375, 5771.56884765625, 5695.4775390625, 6722.8544921875, 6037.232421875, 5852.5419921875, 5543.31884765625, 5279.06396484375, 5672.81005859375, 6319.203125, 6904.9619140625, 7132.4091796875, 6256.390625, 5089.92626953125, 4232.19677734375, 3371.488037109375, 2257.2646484375, 1664.7923583984375, 810.7589721679688, 543.8692016601562, 282.5393981933594, -6.813086032867432, 215.09780883789062, 1100.72509765625, 3266.2685546875, 5211.52978515625, 5998.22607421

### Inference using Boto3 SDK

SageMaker SDK의 `predict()` 메서드로 추론을 수행할 수도 있지만, 이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.<br>
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리
SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다.

참고로 `invoke_endpoint()` 호출을 위한 런타임 클라이언트 인스턴스 생성 시, 로컬 배포 모드에서는 `sagemaker.local.LocalSagemakerRuntimeClient()`를 호출해야 합니다.

In [10]:
client = sagemaker.local.LocalSagemakerClient()
runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = local_model.endpoint_name

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = json.loads(response['Body'].read().decode())

algo-1-71m4l_1  | 2020-08-31 14:00:26,995 [INFO ] W-9000-model com.amazonaws.ml.mms.wlm.WorkerThread - Backend response time: 26
algo-1-71m4l_1  | 2020-08-31 14:00:26,995 [INFO ] W-9000-model ACCESS_LOG - /172.18.0.1:47072 "POST /invocations HTTP/1.1" 200 28


In [11]:
print(outputs)

{'out': [557.2713012695312, 603.84033203125, 355.400146484375, 594.2640380859375, 1020.9503173828125, 2260.816162109375, 5223.166015625, 5095.93505859375, 4293.2744140625, 4693.7001953125, 4462.06982421875, 4440.93359375, 4481.41357421875, 4553.86474609375, 5249.0595703125, 7149.01611328125, 7378.599609375, 6582.26708984375, 5148.14111328125, 3448.327392578125, 2328.8837890625, 2421.5009765625, 1240.4501953125, 969.880615234375, 356.9651184082031, 410.36956787109375, 498.94891357421875, 158.7935333251953, 1257.3603515625, 3725.76708984375, 5771.56884765625, 5695.4775390625, 6722.8544921875, 6037.232421875, 5852.5419921875, 5543.31884765625, 5279.06396484375, 5672.81005859375, 6319.203125, 6904.9619140625, 7132.4091796875, 6256.390625, 5089.92626953125, 4232.19677734375, 3371.488037109375, 2257.2646484375, 1664.7923583984375, 810.7589721679688, 543.8692016601562, 282.5393981933594, -6.813086032867432, 215.09780883789062, 1100.72509765625, 3266.2685546875, 5211.52978515625, 5998.22607421

### Local Mode Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.

In [12]:
def delete_endpoint(client, endpoint_name):
    response = client.describe_endpoint_config(EndpointConfigName=endpoint_name)
    model_name = response['ProductionVariants'][0]['ModelName']

    client.delete_model(ModelName=model_name)    
    client.delete_endpoint(EndpointName=endpoint_name)
    client.delete_endpoint_config(EndpointConfigName=endpoint_name)    
    
    print(f'--- Deleted model: {model_name}')
    print(f'--- Deleted endpoint: {endpoint_name}')
    print(f'--- Deleted endpoint_config: {endpoint_name}')    
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: mxnet-inference-2020-08-31-13-59-38-831
--- Deleted endpoint: local-endpoint-traffic-volume-forecast-1598882374
--- Deleted endpoint_config: local-endpoint-traffic-volume-forecast-1598882374


<br>

## 4. SageMaker Hosted Endpoint Inference
---

이제 실제 운영 환경에 엔드포인트 배포를 수행해 보겠습니다. 로컬 모드 엔드포인트와 대부분의 코드가 동일하며, 모델 아티팩트 경로(`model_data`)와 인스턴스 유형(`instance_type`)만 변경해 주시면 됩니다. SageMaker가 관리하는 배포 클러스터를 프로비저닝하는 시간이 소요되기 때문에 추론 서비스를 시작하는 데에는 약 5~10분 정도 소요됩니다.

In [13]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
role = sagemaker.get_execution_role()
bucket = sagemaker.Session().default_bucket()

In [14]:
model_path = os.path.join(s3_model_dir, "model.tar.gz")
endpoint_name = "endpoint-traffic-volume-forecast-{}".format(int(time.time()))

In [15]:
model = MXNetModel(model_data=model_path,
                         role=role,
                         source_dir='src',
                         entry_point='inference.py',
                         framework_version='1.6.0',
                         py_version='py3')

predictor = model.deploy(instance_type="ml.c5.large", 
                         initial_instance_count=1, 
                         endpoint_name=endpoint_name,
                         wait=True)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------!

추론을 수행합니다. 로컬 모드의 코드와 동일합니다.

In [16]:
import boto3
client = boto3.client('sagemaker')
runtime_client = boto3.client('sagemaker-runtime')
endpoint_name = model.endpoint_name

In [17]:
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/json',
    Accept='application/json',
    Body=json.dumps(input_data)
    )
outputs = json.loads(response['Body'].read().decode())

In [18]:
outputs

{'out': [557.2713012695312,
  603.84033203125,
  355.400146484375,
  594.2640380859375,
  1020.9503173828125,
  2260.816162109375,
  5223.166015625,
  5095.93505859375,
  4293.2744140625,
  4693.7001953125,
  4462.06982421875,
  4440.93359375,
  4481.41357421875,
  4553.86474609375,
  5249.0595703125,
  7149.01611328125,
  7378.599609375,
  6582.26708984375,
  5148.14111328125,
  3448.327392578125,
  2328.8837890625,
  2421.5009765625,
  1240.4501953125,
  969.880615234375,
  356.9651184082031,
  410.36956787109375,
  498.94891357421875,
  158.7935333251953,
  1257.3603515625,
  3725.76708984375,
  5771.56884765625,
  5695.4775390625,
  6722.8544921875,
  6037.232421875,
  5852.5419921875,
  5543.31884765625,
  5279.06396484375,
  5672.81005859375,
  6319.203125,
  6904.9619140625,
  7132.4091796875,
  6256.390625,
  5089.92626953125,
  4232.19677734375,
  3371.488037109375,
  2257.2646484375,
  1664.7923583984375,
  810.7589721679688,
  543.8692016601562,
  282.5393981933594,
  -6.813

### SageMaker Hosted Endpoint Clean-up

엔드포인트를 계속 사용하지 않는다면, 불필요한 과금을 피하기 위해 엔드포인트를 삭제해야 합니다. 
SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있으며, UI에서도 쉽게 삭제할 수 있습니다.

In [19]:
delete_endpoint(client, endpoint_name)

--- Deleted model: mxnet-inference-2020-08-31-14-00-58-618
--- Deleted endpoint: endpoint-traffic-volume-forecast-1598882453
--- Deleted endpoint_config: endpoint-traffic-volume-forecast-1598882453
